In [ ]:
from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression
import numpy as np
from hmmlearn import hmm
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
import statsmodels.api as sm
import seaborn as sns


In [ ]:
# def  model_function(train_data -> np.array, num_of_predictions -> int):

# 	Return prediction 

In [ ]:
googl = yf.download("GOOGL", start="2018-01-01", end="2021-12-31").reset_index()
googl.Date = pd.to_datetime(googl.Date)

In [ ]:
googl

In [ ]:
daily_return = googl['Return'] = np.log(googl.Close /googl.Close.shift(-1)).dropna()
#(np.log(googl.Close /googl.Close.shift(-1))).dropna()
# np.log(googl.Close.pct_change()).dropna()

In [ ]:
plt.plot(daily_return)

In [ ]:
googl[googl.Date>='2021-01-01']

In [ ]:
train,test = googl[googl.Date<'2021-01-01'].Return,googl[googl.Date>='2021-01-01'].Return
# train,test = daily_return[:2800],daily_return[-238:]

In [ ]:
train_np = np.array(train).reshape(-1,1)
test_np = np.array(test[:-1]).reshape(-1,1)

# HMM 

In [ ]:
def hmm_prediction_MCMC(train_data,num_of_predictions,num_samples = 100):
    
    hmm_prediction_values_list = []
    hmm_prediction_states_list = []
    all_df = pd.DataFrame()
    #print(train_data)
    for i in range(num_samples):
        #print(i)
        
        hmm_model = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=1000)
        hmm_model.fit(train_data)

        predict_state = hmm_model.predict(train_data)

        hmm_prediction = hmm_model.sample(num_of_predictions,random_state = predict_state[-1])
        hmm_prediction_values = hmm_prediction[0]
        hmm_prediction_states = hmm_prediction[1]
        
        hmm_prediction_values_list.append(hmm_prediction_values) 
        hmm_prediction_states_list.append(hmm_prediction_states)
        
        temp_df = pd.DataFrame([np.repeat(i,num_of_predictions),np.arange(num_of_predictions),hmm_prediction_values.squeeze()]).T
        
        all_df = pd.concat([all_df, temp_df])
        
    return hmm_prediction_values_list,hmm_prediction_states_list,all_df

In [ ]:
value_list, states_list,mcmc_df = hmm_prediction_MCMC(train_np,len(test_np))

In [ ]:
mcmc_df.columns = ['repeat','time','value']
mcmc_df

In [ ]:
sns.lineplot(data=mcmc_df,x = 'time',y = 'value',ci = 'sd' )
plt.plot(test_np,color = 'r',alpha=0.5,label = 'True Value')
plt.show()

In [ ]:
np_all = np.array(value_list).squeeze()

In [ ]:
mean_np_all = np_all.mean(axis = 0)
std_np_all = np_all.std(axis = 0)

In [ ]:
mean_np_all

In [ ]:
np.sum((test_np.squeeze()<(mean_np_all+std_np_all))*(test_np.squeeze()>(mean_np_all-std_np_all)))/len(test_np)